In [ ]:
print('Data Cleansing - Tarif Dry Reefer')

In [ ]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, time, datetime
import math

In [ ]:
load_dotenv()
db = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_SERVER')
port = os.getenv('POSTGRES_PORT')

In [ ]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [ ]:
sql = """select * from stg_pelni.master_port mp
where mp.flag_ppss ='Y' and mp.is_deleted =false"""

In [ ]:
# Read Query
df_sql = sqlio.read_sql_query(sql, conn)
df_sql2 = sqlio.read_sql_query(sql, conn)

In [ ]:
df_sql.columns.values

In [ ]:
df_sql = df_sql.loc[:, ['code', 'port_name','id_port']]
df_sql2 = df_sql2.loc[:, ['code', 'port_name', 'id_port']]

In [ ]:
df_sql = df_sql.rename(columns={'code' : 'code_embarkasi', 'port_name' : 'embarkasi', 'id_port' : 'id_embarkasi'})
df_sql2 = df_sql2.rename(columns={'code' : 'code_debarkasi', 'port_name' : 'debarkasi', 'id_port' : 'id_debarkasi'})

In [ ]:
# Read CSV
df = pd.read_csv('data/temp_tarif.csv')
df.head()

In [ ]:
# Change Typedata
df['embarkasi'] = df['embarkasi'].astype(str)
df_sql['embarkasi'] = df_sql['embarkasi'].astype(str)

# Inner Join
merge_data = pd.merge(df_sql, 
                      df, 
                      on ='embarkasi', 
                      how ='inner')

In [ ]:
# Change Typedata
df['debarkasi'] = df['debarkasi'].astype(str)
df_sql2['debarkasi'] = df_sql2['debarkasi'].astype(str)

# Inner Join
cleandf = pd.merge(df_sql2, 
                      merge_data, 
                      on ='debarkasi', 
                      how ='inner')

In [20]:
cleandf.columns.values

array(['code_debarkasi', 'debarkasi', 'id_debarkasi', 'code_embarkasi',
       'embarkasi', 'id_embarkasi', 'id', 'dry', 'reefer'], dtype=object)

In [21]:
cleandf = cleandf.loc[:,['id_embarkasi','id_debarkasi', 'debarkasi', 'embarkasi', 'code_embarkasi', 'code_debarkasi','dry', 'reefer']]
cleandf

,id_embarkasi,id_debarkasi,debarkasi,embarkasi,code_embarkasi,code_debarkasi,dry,reefer
0,cc0756e0-437b-4403-b4c9-14d4bc895620,b456ed3f-543d-416e-98a2-e835bf4f4bec,Amurang,Toli-toli,861,920,"2,451,000","2,884,000"
1,b166ff1e-a3f6-4899-a4f4-27f7ad076b6a,b456ed3f-543d-416e-98a2-e835bf4f4bec,Amurang,Manokwari,972,920,"7,496,000","8,816,000"
2,3c42cdbb-1dd7-4214-99d3-ffabc2c2a93d,b456ed3f-543d-416e-98a2-e835bf4f4bec,Amurang,Jayapura,974,920,"11,203,000","13,178,000"
3,5267592e-5d29-4fdf-8975-7ade3bd50b39,b456ed3f-543d-416e-98a2-e835bf4f4bec,Amurang,Pantoloan,863,920,"4,465,000","5,253,000"
4,c2f757c2-442b-4fce-9a53-4292baf0f776,b456ed3f-543d-416e-98a2-e835bf4f4bec,Amurang,Semayang,803,920,"5,920,000","6,964,000"
...,...,...,...,...,...,...,...,...
448,55596cb8-7e4e-4b06-b557-bee57775e70a,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,Bau-Bau/Murhum,921,251,"12,379,000","14,561,000"
449,6a01e7c5-05e6-4ab7-a706-f0936f32a837,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,Makassar,893,251,"10,625,000","12,498,000"
450,19887a24-9011-425f-bc0c-5f5cd512ba5a,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,Tanjung Perak,563,251,"7,375,000","8,676,000"
451,7d09ed71-36f8-40bf-996a-5a09bae5dc3d,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,Bitung,835,251,"17,741,000","20,868,000"
